This notebook explores the new metadata I got from Rachel this morning at Longwood.

We were prompted to get these data from reviewers requesting more symptoms, medications, and demographics metadata. I collated all the patients we used (in [2019-01-21.collate_all_patients.ipynb](2019-01-21.collate_all_patients.ipynb)) and sent that to her so her techs could prepare metadata files with cleaner metadata.

This notebook is a mess and involves a few back-and-forths with the team. I'll make a different notebook or script to clean up the data...

In [1]:
import pandas as pd
import numpy as np

In [2]:
# This was the respiratory study, so these patients have more extensive
# respiratory symptoms and they also will have BAL samples
fk23_v1 = '../../data/raw/metadata/reviewer-response-jan2019/k23 data.csv'
fk23_v2 = '../../data/raw/metadata/reviewer-response-jan2019/FINAL AIM 2, 3, 4 BASELINE DATA.csv'

# R01 study is different, and focuses more on general infections. So it has
# more metadata on infections but less on respiratory symptoms. Also, these
# patients probably won't have BAL.
r01base_v1 = '../../data/raw/metadata/reviewer-response-jan2019/baselineR01.csv'
r01long = '../../data/raw/metadata/reviewer-response-jan2019/r01 longitudinal.csv'
r01base_v2 = '../../data/raw/metadata/reviewer-response-jan2019/baselineforclaire-r01-aim3supp.csv'

# Fundoplication patients
fundo = '../../data/raw/metadata/reviewer-response-jan2019/fundobaselinedata_1.csv'

dfk23_v1 = pd.read_csv(fk23_v1)
dfk23_v2 = pd.read_csv(fk23_v2)
dfr01base_v1 = pd.read_csv(r01base_v1)
dfr01base_v2 = pd.read_csv(r01base_v2)
dfr01long = pd.read_csv(r01long)
dffundo = pd.read_csv(fundo)

alldfs = [dfk23_v1, dfk23_v2, dfr01base_v1, dfr01base_v2, dfr01long, dffundo]

print([i.shape for i in alldfs])

[(154, 236), (156, 236), (16, 203), (23, 369), (22, 369), (26, 529)]


In [3]:
# Figure out subject ID column label for each
print([i.columns[0] for i in alldfs])

['SUBJID', 'SUBJID', 'SUBJID', 'SUBJID', 'SUBJID', 'SUBJID_BL']


Replace the one different subject ID, add a label indicating which metadata the data came from, and concatenate.

In [4]:
dffundo = dffundo.rename(columns={'SUBJID_BL':"SUBJID"})
dffundo.head()

,SUBJID,SUBJINTL_BL,CLAIREDATA,COMPDATE_BL,PPIstatus_BL,INITIAL_BL,ANSWER_BL,ANSWERSP_BL,GENDER_BL,DOB_BL,...,DATEFUNDO,PHMII,DATEPHMII,ABXINPT,ABX1SP,ABX1DUR,ABX1DOSE,ABX2SP,ABX2DUR,ABX2DOSE
0,021-7 F1,EC,1,,0,,,,0,7/24/2005,...,,,,,,,,,,
1,022-3-F1,WRY,1,7/24/2012,1,ARY,2,-6.00,0,9/22/2008,...,7/24/2012,0,,0,-6,-6,-6,-6,-6,-6
2,024-1-F1,MLW,1,8/2/2012,1,-9,2,-6.00,1,5/18/2002,...,,,,,,,,,,
3,025-6-F1,SP,1,8/22/2012,1,CAJ,2,-6.00,1,11/14/2004,...,,,,,,,,,,
4,026-8-F1,BA,1,,1,,,,,10/25/2009,...,,,,,,,,,,


In [5]:
c = 'metadata_file'
c2 = 'metadata_id'

dfk23_v1[c] = fk23_v1.rsplit('/', 1)[1]
dfk23_v1[c2] = 'k23_v1'

dfk23_v2[c] = fk23_v2.rsplit('/', 1)[1]
dfk23_v2[c2] = 'k23_v2'

dfr01base_v1[c] = r01base_v1.rsplit('/', 1)[1]
dfr01base_v1[c2] = 'r01_baseline_v1'

dfr01base_v2[c] = r01base_v2.rsplit('/', 1)[1]
dfr01base_v2[c2] = 'r01_baseline_v2'

dfr01long[c] = r01long.rsplit('/', 1)[1]
dfr01long[c2] = 'r01_longitudinal'

dffundo[c] = fundo.rsplit('/', 1)[1]
dffundo[c2] = 'fundo'

In [6]:
# Check that the list of df's is also updated
alldfs = [dfk23_v1, dfk23_v2, dfr01base_v1, dfr01base_v2, dfr01long, dffundo]

In [7]:
bigdf = pd.concat(alldfs, sort=False)
print(bigdf.shape, bigdf['SUBJID'].dropna().shape, bigdf['SUBJID'].unique().shape)

((397, 1157), (397,), (222,))


# Check that these metadata files have all the patients I need

In [8]:
# Compare with the patients I used in the analyses
fpatients = '../../final/supp_files/combined_patient_site_and_analyses.csv'
dfpts = pd.read_csv(fpatients, index_col=0)
dfpts.head()

,bal,gastric_fluid,throat_swab,stool,figure1.overview_plots,figure2.2016_pcoa,figure2.2014_pcoa,figure2.between_patient_jsd_2016_permanova,figure2.between_patient_jsd_2014_permanova,figure2.between_patient_jsd,...,aspiration_classifiers_exchanged_OTUs.bal_gastric_fluid,aspiration_classifiers_exchanged_OTUs.bal_throat_swab,aspiration_classifiers.bal,aspiration_classifiers.gastric_fluid,aspiration_classifiers.throat_swab,aspiration_classifiers.bal_throat_swab,aspiration_classifiers.bal_gastric_fluid,aspiration_classifiers.throat_swab_gastric_fluid,aspiration_classifiers.bal_throat_swab_gastric_fluid,figure5.reflux_correlation
04-080-7,True,True,True,False,True,False,True,False,True,True,...,False,False,False,False,False,False,False,False,False,False
01-299-7,False,False,True,False,False,True,False,True,False,True,...,False,False,False,False,False,False,False,False,False,False
04-167-8,False,True,False,False,False,False,True,False,True,True,...,False,False,False,True,False,False,False,False,False,False
04-087-1,True,True,True,False,True,False,True,False,True,True,...,False,False,False,False,False,False,False,False,False,False
042-6-F1,False,False,True,False,False,False,True,False,True,True,...,False,False,False,False,False,False,False,False,False,False


## Find missing patients

We'll do duplicates next.

In [9]:
newmeta_allsubjs = [i['SUBJID'].tolist()[:] for i in alldfs]
newmeta_allsubjs = [s for i in newmeta_allsubjs for s in i]

print(len(newmeta_allsubjs), len(set(newmeta_allsubjs)))

(397, 222)


In [10]:
analysis_allsubjs = dfpts.index.tolist()

In [11]:
print(len([i for i in analysis_allsubjs if i not in newmeta_allsubjs]))

13


13! That seems like a manageable number! Margot's email had a lot of caveats, let's see how many match up...

In [12]:
missing_subjs = [i for i in analysis_allsubjs if i not in newmeta_allsubjs]
#print('\n'.join(missing_subjs))

```
04-168-4 - "04-168-4 is listed in the database as 14-168-4"
021-7-F1
14-233-0 - "14-233-0 is listed in the database as 04-233-1"
03-125-1 - "03-125-9 (we believe this was listed as 03-125-1 but the correct ID should be 03-125-9 based on our sample logs)"
04-164-1 - "04-164-1 is listed in the database as 02-164-1"
045-9-F1 - 
04-235-8 - "04-235-8 is listed in the database as 04-235-4"
13-089-1 - "13-089-1 - listed in the "r01" database as 01-089-1"
046-2-F1
01-297-4 - "01-297-4 is actually 01-297-2 - this is in the "r01" database"
13-058-2 - "13-058-2 - listed in the "r01" database as 01-058-2"
029-6-F1 - "please note that 029-6 is listed in database as 029-5"
dup05-013-1
```

I wonder if these missing fundo patients have something wonky with the leading zeros?

In [13]:
[i for i in newmeta_allsubjs if i.endswith('F1')]

['021-7 F1',
 '022-3-F1',
 '024-1-F1',
 '025-6-F1',
 '026-8-F1',
 '027-4-F1',
 '028-1-F1',
 '029-5-F1',
 '030-5-F1',
 '032-1-F1',
 '033-8-F1',
 '035-1-F1',
 '036-2-F1',
 '037-3-F1',
 '040-5-F1',
 '041-3-F1',
 '042-6-F1',
 '043-7-F1',
 '045-9 F1',
 '046-2 F1',
 '048-1-F1',
 '050-4-F1',
 '051-6-F1',
 '055-1-F1',
 '057-9-F1',
 '058-4-F1']

Aha! A few of the fundo samples are missing the dash!

In [14]:
dffundo['SUBJID'] = dffundo['SUBJID'].str.replace(' ', '-')
bigdf['SUBJID'] = bigdf['SUBJID'].str.replace(' ', '-')

In [15]:
newmeta_allsubjs = [i['SUBJID'].tolist()[:] for i in alldfs]
newmeta_allsubjs = [s for i in newmeta_allsubjs for s in i]

missing_subjs = [i for i in analysis_allsubjs if i not in newmeta_allsubjs]

print(len(missing_subjs))

10


Good, now it's just the ten patients Margot described and the dup sample (which I may have forgotten to send them...)

In [16]:
'05-013-1' in newmeta_allsubjs

False

Wow I'm super tired. Let's call it a night.

**TO DO**

- check that I know which metadata column to use in all of the files
- email Margot asking for metadata for 05-013-1 (I accidentally didn't copy that one), and maybe column info if needed
- find and reconcile duplicates
    - if all the metadata columns match, just collapse
    - if not, pick the metadata from more recent files

## Check columns

Pneumonia: history of pneumonia, combine both datasets

Antibiotics: use the 1 month mark, say it's a "recent history of abx"
--> note: this is all parent report

PPI = on PPIs now (PPI48HRS and MEDI2C)

Inhaled steroids: compare the 2 months from R01 and current steroids from the K23 data
	MEDI3C and INHAL2MOS

Oral steroids: MEDI4C in K23 and (SWALL2MOS or ORAL2MOS) in the R01

Other symptoms: use the 6 months that are common between the two datasets

### Metadata columns I am using (to double check, esp fundoplication)

* History of pneumonia
    - K23: PNEUMN
    - R01: T0.PNEUM
    - fundo: pna6mo_BL, hxofPNA_BL, or combination (if it's yes in either then, it's a yes overall)?
* Recent history of antibiotics
    - K23: ANTIB1MO
    - R01: T0ABX1MO
    - Fundo: ANTIB1MONTH -- there are other antibiotics columns, but I am guessing these are related to the post-fundo samples? e.g. ANTIBIO1, ANTIBIO1SP, ANTIBIO3, ANTIBO3SP, ANTIBIO6
* On PPIs now
    - K23: MEDI2C
    - R01: PPI48HRS
    - fundo: PPIstatus_BL, other otpions: ppi6mo_BL, Y1ppi_BL
* Recent usage of inhaled steroids
    - K23: MEDI3C
    - R01: INHAL2MOS
    - fundo: inhsteroid6mo_BL (update: it's actually MEDI3C_BL)
        - **NOTE**: do fundoplication samples have data on inhaled steroids more recent than 6 months? [yes, MEDI3C_BL]
* Recent use of oral steroids
    - K23: MEDI4C 
    - R01: SWALL2MOS or ORAL2MOS (i.e. if it's either one of those, then it's a yes)
    - Fundo: swallster6mo_BL (update: this should be MEDI4C_BL)
        - **NOTE**: do fundoplication samples have data on swallowed steroids more recent than 6 months? (yes, MEDI4C_BL)

For the symptoms, can you send me the full description? i.e. the text that I should put in the Table.

* Problem swallowing
    - K23 and R01: PSWALL
    - fundo: pswall6mo_BL
* Food stuck (what should I call this one?)
    - K23 and R01: FOODSTUCK
    - Fundo: foodstuck6mo_BL
* Difficulty swallowing
    - K23 and R01: SWLLDIFF
    - Fundo: diffswall6mo_BL
* Abdominal pain
    - K23 and R01: ABPAIN
    - Fundo: abdpain6mo_BL
* Constipation
    - K23 and R01: CONSTIP
    - Fundo: ??
* Weight loss
    - K23 and R01: WTLOSS
    - Fundo: wtloss6mo_BL
* Food up (what should I call this one?)
    - K23 and R01: FOODUP
    - Fundo: foodup6mo_BL
* Chest pain
    - K23 and R01: CHSTPAIN
    - Fundo: chstpain6mo_BL
* Asthma
    - K23 and R01: ASTHMA
    - Fundo: asthma6mo_BL
* Chronic cough
    - K23 and R01: CHRONCOUGH
    - Fundo: chroncoughhx_BL (update: should be COND48_BL)

---

Update from Margot's email on 1/28:

See below the different metadata columns for the FUNDO database. The rest you have the correct column names.

a.       * History of pneumonia: pna6mo_BL is if had pneumonia in past 6month. hxofPNA_BL: is if subject had history of pneumonia at any point in life. I would use the 6month one since the other databases includes that time period.

b.      * Recent history of antibiotics: ANTIB1MONTH

c.       * On PPIs now: PPIstatus_BL

d.      * Recent usage of inhaled steroids: MEDI3C_BL: currently taking inhaled steroids

e.      * Recent use of oral steroids: MEDI4C_BL :currently taking swallowed steroids

f.        * Constipation: FUNDO database does not have this information

g.       * Chronic cough: COND48_BL : this is for the past 6 months.

In [17]:
bigdf.index = bigdf['SUBJID']

In [18]:
# Pneumonia

# PENUMN = K23 data
# T0.PNEUM = R01 data
# Not sure about fundo, looks like maybe pna6mo_BL (or hxofPNA_BL?)

bigdf.shape, bigdf[['PNEUMN', 'T0.PNEUM', 'pna6mo_BL', 'hxofPNA_BL', 'metadata_id']].dropna(how='all', axis=1).shape

((397, 1157), (397, 5))

In [19]:
# Antibiotics, 1 months

# ANTIB1MO = K23 data
# T0ABX1MO = R01 data
# Fundo: ANTIB1MONTH, (there are other antibiotics columns, 
# but I am guessing these are related to the post-fundo samples? 
# e.g. ANTIBIO1, ANTIBIO1SP, ANTIBIO3, ANTIBO3SP, ANTIBIO6)
bigdf.shape, bigdf[['ANTIB1MO', 'T0ABX1MO', 'ANTIB1MONTH', 'metadata_id']].dropna(how='all', axis=1).shape

((397, 1157), (397, 4))

In [20]:
# On PPIs now

# K23: MEDI2C
# R01: PPI48HRS
# fundo: PPIstatus_BL, other otpions: ppi6mo_BL, Y1ppi_BL

bigdf.shape, bigdf[['MEDI2C', 'PPI48HRS', 'PPIstatus_BL', 'metadata_id']].dropna(how='all', axis=1).shape

((397, 1157), (397, 4))

In [21]:
# Recent history of inhaled steroids

# K23: MEDI3C
# R01: INHAL2MOS
# fundo: inhsteroid6mo_BL
bigdf.shape, bigdf[['MEDI3C', 'INHAL2MOS', 'inhsteroid6mo_BL', 'metadata_id']].dropna(how='all', axis=1).shape

((397, 1157), (397, 4))

In [22]:
# Oral steroids

# K23: MEDI4C 
# R01: SWALL2MOS or ORAL2MOS (i.e. if it's either one of those, then it's a yes)
# Fundo: swallster6mo_BL

bigdf.shape, bigdf[['MEDI4C', 'SWALL2MOS', 'ORAL2MOS', 'swallster6mo_BL', 'metadata_id']].dropna(how='all', axis=1).shape

((397, 1157), (397, 5))

## Find duplicate patients

In [26]:
# Which patients are in there twice?
from collections import Counter

n_duplicates = pd.Series(Counter(newmeta_allsubjs))
dup_subjs = n_duplicates[n_duplicates > 1].index.tolist()

In [27]:
dup_df = bigdf.loc[dup_subjs, ['SUBJID', 'metadata_id']]

In [28]:
dup_df.groupby(['metadata_id']).size()

metadata_id
k23_v1              154
k23_v2              154
r01_baseline_v2      20
r01_longitudinal     21
dtype: int64

In [29]:
tmpdict = {}
for s, subdf in dup_df.groupby('SUBJID'):
    s_meta = '-'.join(subdf['metadata_id'].tolist())
    tmpdict[s] = s_meta

/Users/claire/anaconda/envs/aspiration/lib/python2.7/site-packages/ipykernel_launcher.py:2: FutureWarning: 'SUBJID' is both an index level and a column label.
Defaulting to column, but this will raise an ambiguity error in a future version
  


In [30]:
tmp = pd.DataFrame(tmpdict, index=['metadata_str']).T
tmp.groupby(['metadata_str']).size()

metadata_str
k23_v1-k23_v2                                        154
r01_baseline_v2-r01_longitudinal                      19
r01_baseline_v2-r01_longitudinal-r01_longitudinal      1
dtype: int64

So 154 subjects have metadata in both k23's, 19 have it in both r01_baseline_v2 and r01_longitudinal and one has it twice in r01_longitudinal plus r01_baseline_v2.

Let's remove the first K23 data from this analysis entirely.

In [31]:
dfr01long.shape, dfr01base_v2.shape

((22, 371), (23, 371))

In [32]:
# Just go through and grab all the idx's we want to remove

# Reset index to get integer indexing (rather than subject ID, which has duplicates)
bigdf = bigdf.reset_index(drop=True)

remove_ix = []
for subj, subj_df in bigdf.groupby('SUBJID'):
    if subj_df.shape[0] > 1:
        
        meta_ids = subj_df['metadata_id'].values
        
        if 'k23_v1' in meta_ids and 'k23_v2' in meta_ids:
            rmv_ix = subj_df.query('metadata_id == "k23_v1"').index.tolist()
            remove_ix += rmv_ix
            
        elif 'r01_baseline_v2' in meta_ids and 'r01_longitudinal' in meta_ids:
            rmv_ix = subj_df.query('metadata_id == "r01_longitudinal"').index.tolist()
            remove_ix += rmv_ix
        
        else:
            print(subj)

In [33]:
len(remove_ix)

175

## Remove duplicate patients

In [34]:
bigdf_nodup = bigdf.drop(index=remove_ix)
bigdf_nodup.shape

(222, 1157)

In [35]:
bigdf_nodup.groupby(['SUBJID']).size().max()

1

Okay, we have no more duplicates! Woohoo!

Wait, we have 222 patients total in the study but are still missing one here... (dup05-013-1)

In [36]:
bigdf_nodup.query('SUBJID == "dup05-013-1"')

,SUBJID,CLAIRESEQUENC,SUBJINTL,AIM1YN,AIM2YN,AIM3YN,VISITDT,VISITNO,COMPDATE,INITIAL,...,DATEFUNDO,PHMII,DATEPHMII,ABXINPT,ABX1SP,ABX1DUR,ABX1DOSE,ABX2SP,ABX2DUR,ABX2DOSE


In [37]:
'05-013-1' in bigdf['SUBJID'].values

False

In [38]:
np.sort([i for i in analysis_allsubjs if i not in bigdf_nodup['SUBJID'].tolist()])

array(['01-297-4', '029-6-F1', '03-125-1', '04-164-1', '04-168-4',
       '04-235-8', '13-058-2', '13-089-1', '14-233-0', 'dup05-013-1'],
      dtype='|S11')

```
01-297-4 - "01-297-4 is actually 01-297-2 - this is in the "r01" database"
029-6-F1 - "please note that 029-6 is listed in database as 029-5"
03-125-1 - "03-125-9 (we believe this was listed as 03-125-1 but the correct ID should be 03-125-9 based on our sample logs)"
04-164-1 - "04-164-1 is listed in the database as 02-164-1"
04-168-4 - "04-168-4 is listed in the database as 14-168-4"
04-235-8 - "04-235-8 is listed in the database as 04-235-4"
13-058-2 - "13-058-2 - listed in the "r01" database as 01-058-2"
13-089-1 - "13-089-1 - listed in the "r01" database as 01-089-1"
14-233-0 - "14-233-0 is listed in the database as 04-233-1"

dup05-013-1
```

In [39]:
# Hmm.. what about samples that are in the new metadata but not in my analysis?
np.sort([i for i in bigdf_nodup['SUBJID'].tolist() if i not in analysis_allsubjs])

array(['01-058-2', '01-089-1', '01-199-7', '01-297-2', '029-5-F1',
       '03-125-9', '03-200-1', '04-233-1', '04-235-4', '14-168-4'],
      dtype='|S8')

```
13-058-2 - "13-058-2 - listed in the "r01" database as 01-058-2"
13-089-1 - "13-089-1 - listed in the "r01" database as 01-089-1"
01-297-4 - "01-297-4 is actually 01-297-2 - this is in the "r01" database"
029-6-F1 - "please note that 029-6 is listed in database as 029-5"
03-125-1 - "03-125-9 (we believe this was listed as 03-125-1 but the correct ID should be 03-125-9 based on our sample logs)"
14-233-0 - "14-233-0 is listed in the database as 04-233-1"
04-235-8 - "04-235-8 is listed in the database as 04-235-4"
04-168-4 - "04-168-4 is listed in the database as 14-168-4"


04-164-1 - "04-164-1 is listed in the database as 02-164-1"
dup05-013-1
```

Ones I didn't find:

01-199-7    
03-200-1

Ah okay and it looks like 04-164-1 and 02-164-1, which are actually the same patient, are both in my metadata...


In [40]:
dfpts.loc[['04-164-1', '02-164-1']]

,bal,gastric_fluid,throat_swab,stool,figure1.overview_plots,figure2.2016_pcoa,figure2.2014_pcoa,figure2.between_patient_jsd_2016_permanova,figure2.between_patient_jsd_2014_permanova,figure2.between_patient_jsd,...,aspiration_classifiers_exchanged_OTUs.bal_gastric_fluid,aspiration_classifiers_exchanged_OTUs.bal_throat_swab,aspiration_classifiers.bal,aspiration_classifiers.gastric_fluid,aspiration_classifiers.throat_swab,aspiration_classifiers.bal_throat_swab,aspiration_classifiers.bal_gastric_fluid,aspiration_classifiers.throat_swab_gastric_fluid,aspiration_classifiers.bal_throat_swab_gastric_fluid,figure5.reflux_correlation
04-164-1,False,False,True,False,False,False,True,False,True,True,...,False,False,False,False,False,False,False,False,False,False
02-164-1,False,True,False,False,False,False,True,False,True,True,...,False,False,False,True,False,False,False,False,False,False


Are these aspirators? If not, I don't think it's worth fixing...

In [41]:
#dfpts.columns

In [42]:
dfpts.loc[['04-164-1', '02-164-1'] , 
          ['exchange_calculation.gastric_fluid-throat_swab', 'figure3.within_patient_beta_div', 
           'figure4.asp_vs_nonasp_beta', 'aspiration_classifiers.throat_swab']]

,exchange_calculation.gastric_fluid-throat_swab,figure3.within_patient_beta_div,figure4.asp_vs_nonasp_beta,aspiration_classifiers.throat_swab
04-164-1,False,False,False,False
02-164-1,False,False,False,False


No, I think I can leave these two as they are, since they aren't used in the most important parts of the analysis or the ones with low N. 

Having samples that seem like they're from two different people when they're actually the same person would only affect my between-patient calculations, but pulling them all away from my conclusions (e.g. it would make it seem like gastric/throat between people are more similar than they actually are.)

I just realized that the dup05 patient shouldn't be in these analyses, because they're a lung transplant patient... What analyses did I use this patient in?

In [43]:
dfpts.loc['dup05-013-1']

bal                                                                   True
gastric_fluid                                                         True
throat_swab                                                          False
stool                                                                False
figure1.overview_plots                                               False
figure2.2016_pcoa                                                    False
figure2.2014_pcoa                                                     True
figure2.between_patient_jsd_2016_permanova                           False
figure2.between_patient_jsd_2014_permanova                            True
figure2.between_patient_jsd                                           True
figure3.site_classifiers                                              True
figure3.within_patient_beta_div                                       True
figure4.asp_vs_nonasp_beta                                           False
exchange_calculation.bal-

Okay, this one is fine too. We'll just keep it in the data but not have any metadata for it. I think that's okay - again, shouldn't really affect any of our results.

# Gender and age

In [47]:
gender_cols = ['GENDER', 'GENDER_BL']
#[i for i in bigdf_nodup.columns if 'gender' in i.lower()]
bigdf_nodup.shape, bigdf_nodup[gender_cols].dropna(how='all').shape

((222, 1157), (222, 2))

Any patients missing gender data?

In [50]:
bigdf_nodup[bigdf_nodup['GENDER'] == '-6']

,SUBJID,CLAIRESEQUENC,SUBJINTL,AIM1YN,AIM2YN,AIM3YN,VISITDT,VISITNO,COMPDATE,INITIAL,...,DATEFUNDO,PHMII,DATEPHMII,ABXINPT,ABX1SP,ABX1DUR,ABX1DOSE,ABX2SP,ABX2DUR,ABX2DOSE
170,04-062-8,1.0,TS,0,1,0,3/8/2011,1,3/8/2011,,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [52]:
bigdf_nodup[bigdf_nodup['GENDER_BL'] == ' ']

,SUBJID,CLAIRESEQUENC,SUBJINTL,AIM1YN,AIM2YN,AIM3YN,VISITDT,VISITNO,COMPDATE,INITIAL,...,DATEFUNDO,PHMII,DATEPHMII,ABXINPT,ABX1SP,ABX1DUR,ABX1DOSE,ABX2SP,ABX2DUR,ABX2DOSE
375,026-8-F1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,,,,,,,,,,


In [51]:
bigdf_nodup['GENDER_BL'].unique()

array([nan, '0', '1', ' ', '2'], dtype=object)